In [ ]:


# Define geometries (esferas y plancha)
def create_geometries():

    # Obstrucción (si hay alguna, por ejemplo, otra esfera)
    z_translation, r = 0.5, 0.1
    
    obstacle = pv.Circle(radius=r, resolution=10)
    obstacle.translate([0, 0, z_translation], inplace=True)
    
    # Define line segment
    start = rectangle_down.cell_centers().points[0]
    stop = rectangle_up.cell_centers().points[0]
    # Perform ray trace
    points, ind = obstacle.ray_trace(start, stop)

    # Create geometry to represent ray trace
    ray = pv.Line(start, stop)
    intersection = pv.PolyData(points)

    return [obstacle, ray, intersection, rectangle_down, rectangle_up]

obstacle, ray, intersection, rectangle_down, rectangle_up = create_geometries()

F = compute_viewfactor(rectangle_down,rectangle_up)

pvf.get_visibility(rectangle_up , rectangle_down)



(False, '')

In [164]:
import pyvista as pv
import numpy as np
import pyviewfactor as pvf 
from pyviewfactor import get_visibility_raytrace
from pyviewfactor import  compute_viewfactor, fc_unstruc2poly

Altura = 6678000
Radio = 6378000

In [ ]:
# Plancha (línea en 3D)

pointa = [0, Altura - 1, -1]
pointb = [0, Altura + 1, 1]
pointc = [0, Altura + 1, -1]
    
liste_pts = [pointa, pointb, pointc]
liste_pts.reverse()
    
rectangle_normal = pv.Rectangle([pointa, pointb, pointc])
rectangle_inverso = pv.Rectangle(liste_pts)

esfera_satelite = pv.SolidSphere(outer_radius = 1, center=(-1.5, Altura, 0), start_theta=0, end_theta=360)
esfera_satelite_surface = esfera_satelite.extract_surface()

start = [0, 0, 0]  # Centro de la Tierra
end = [0, Altura, 0]    # Centro del satélite
flecha = pv.Arrow(start=start, direction=[end[0] - start[0], end[1] - start[1], end[2] - start[2]], scale=0.5)

# create a raw sphere with pyvista
Tierra = pv.Sphere(radius = Radio, center=(0, 0, 0), direction=(0, 0, 1), theta_resolution=50, phi_resolution=50)

C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


In [167]:
# "one array to contain them all" -> the results will be stored there
F = np.zeros(sphere.n_cells)

for i in range(sphere.n_cells):
    circular_face = sphere.extract_cells(i)
    circular_face = fc_unstruc2poly(circular_face) # convert to PolyData
    
    if get_visibility_raytrace(rectangle_up, circular_face, rectangle_down) :
        F[i] = compute_viewfactor(circular_face, rectangle_up)
    else:
        F[i] = 0.0
    

KeyboardInterrupt: 

In [116]:
# put the scalar values in the geometry
sphere.cell_data["F"] = F
sphere.save("./sphere.vtk") # ... and save.

In [212]:
# Render the result
p = pv.Plotter(notebook=True)
p.add_mesh(rectangle_normal, show_edges=True, opacity=0.5, color="red", lighting=False, label="rectangle")
p.add_mesh(Tierra, color="yellow", line_width=5,opacity=0.5, label="rect down")
p.add_mesh(esfera_satelite_surface, color="green", line_width=5, label="ray trace")
#p.add_mesh(sphere, scalars='F', cmap='jet') # add mesh with a nice color scheme

p.show()

Widget(value='<iframe src="http://localhost:56068/index.html?ui=P_0x22d35f13e10_69&reconnect=auto" class="pyvi…